In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
# Install all dependencies
!pip install fastapi uvicorn pyngrok nest_asyncio diffusers transformers accelerate python-multipart chromadb langchain-community langchain-core


In [9]:
import os
import shutil
from pyngrok import ngrok, conf

# 1. NUKE THE OLD CONFIG (The fix)
# This deletes the hidden file where Kaggle stores the old token
ngrok_path = os.path.join(os.path.expanduser("~"), ".ngrok2")
if os.path.exists(ngrok_path):
    shutil.rmtree(ngrok_path)
    print("🗑️ Deleted old Ngrok config cache.")

# 2. KILL STUCK PROCESSES
os.system("killall ngrok")
os.system("fuser -k 8000/tcp")

# 3. SET NEW TOKEN (Explicitly)
# Go to dashboard.ngrok.com and verify this is 100% fresh
NEW_TOKEN = "36YOOzHxWv0GpRmY9IeEhc1C2h0_LgQ8wA5MsJCn8hSteyw6" 

# Create a clean config
conf.get_default().auth_token = NEW_TOKEN
conf.get_default().region = "us" # Force region to US to avoid connection issues

# 4. CONNECT
# We pass the token directly here to bypass any config file issues
try:
    public_url = ngrok.connect(8000, pyngrok_config=conf.get_default()).public_url
    print(f"\n✅ SUCCESS! URL: {public_url}")
except Exception as e:
    print(f"\n❌ ERROR: {e}")


✅ SUCCESS! URL: https://overclose-georgann-soporifically.ngrok-free.dev


In [1]:
# ==========================================
# PART A: THE LANGCHAIN RAG BACKEND
# ==========================================
import os
import threading
import io
import cv2
import numpy as np
import requests
import torch
import chromadb
from PIL import Image
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# LangChain Imports
from langchain_community.vectorstores import Chroma
from langchain_core.embeddings import Embeddings
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import Response

# Models
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from transformers import CLIPProcessor, CLIPModel

# --- 1. SETUP & CLEANUP ---
import os
from pyngrok import ngrok, conf

# Kill any existing servers to free the port
try:
    os.system("fuser -k 8000/tcp")
except:
    pass

# PASTE YOUR NEW TOKEN HERE
NGROK_TOKEN = "36YOOzHxWv0GpRmY9IeEhc1C2h0_LgQ8wA5MsJCn8hSteyw6" 

# Force update the configuration
# This overwrites any old cached token that caused the error
conf.get_default().auth_token = NGROK_TOKEN
ngrok.set_auth_token(NGROK_TOKEN)

# Select Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Device selected: {device}")

# --- 2. CUSTOM CLIP EMBEDDINGS (The Resume Flex) ---
# LangChain usually expects text-only embeddings (like OpenAI).
# We build a CUSTOM class to let LangChain use our CLIP model.
class CLIPEmbeddings(Embeddings):
    def __init__(self):
        self.device = device
        self.model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(self.device)
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    def embed_documents(self, texts):
        # Not used for this RAG (we embed images directly), but required by class
        return self.embed_query(texts[0])

    def embed_query(self, text):
        inputs = self.processor(text=[text], return_tensors="pt", padding=True).to(self.device)
        with torch.no_grad():
            features = self.model.get_text_features(**inputs)
        return (features / features.norm(p=2, dim=-1, keepdim=True)).cpu().numpy().tolist()[0]

    def embed_image(self, image_path):
        image = Image.open(image_path).convert("RGB")
        inputs = self.processor(images=image, return_tensors="pt").to(self.device)
        with torch.no_grad():
            features = self.model.get_image_features(**inputs)
        return (features / features.norm(p=2, dim=-1, keepdim=True)).cpu().numpy().tolist()[0]

# Initialize our custom embeddings
clip_embeddings = CLIPEmbeddings()

# --- 3. BUILD LANGCHAIN VECTORSTORE ---
print("Building LangChain VectorStore...")

# Download Images (you can add images here to expand knowledge database)

style_data = {
    "Cyberpunk": "https://images.unsplash.com/photo-1515036551567-bf1198cccc35?q=80&w=2076&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D",
    "Ghibli": "https://images.unsplash.com/photo-1630207831419-3532bcb828d7?q=80&w=1931&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D",
    "Sketch": "https://plus.unsplash.com/premium_vector-1711987527538-4ff22f0fdbbc?q=80&w=1922&auto=format&fit=crop&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D",
    "Industrial": "https://images.unsplash.com/photo-1505577058444-a3dab90d4253?w=600"
}

os.makedirs("styles", exist_ok=True)
db_client = chromadb.Client()
try: db_client.delete_collection("langchain_styles")
except: pass

collection = db_client.create_collection("langchain_styles")

# Manually ingest because LangChain's default ingestion is text-only
for name, url in style_data.items():
    path = f"styles/{name}.jpg"
    if not os.path.exists(path):
        Image.open(requests.get(url, stream=True).raw).save(path)

    # We store the IMAGE embedding, but LangChain will search using TEXT
    emb = clip_embeddings.embed_image(path)
    collection.add(ids=[name], embeddings=[emb], metadatas=[{"path": path}], documents=[name])
    print(f"Ingested: {name}")

# Create the LangChain Retriever interface
vectorstore = Chroma(
    client=db_client,
    collection_name="langchain_styles",
    embedding_function=clip_embeddings
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# --- 4. GENERATION PIPELINE (The Tool) ---
print("Loading SD Pipeline...")
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
).to('cuda')
pipe.load_ip_adapter("h94/IP-Adapter", subfolder="models", weight_name="ip-adapter_sd15.bin")

def generation_node(inputs):
    """
    This is a 'RunnableLambda' - a function that acts as a step in the chain.
    """
    prompt = inputs["prompt"]
    style_path = inputs["style_path"]
    sketch = inputs["sketch"] # PIL Image

    print(f"🎨 Chain generating with style: {style_path}")

    # 1. Load Style Image
    style_image = Image.open(style_path)

    # 2. Canny Edge
    image_np = np.array(sketch)
    image_np = cv2.Canny(image_np, 50, 150)
    image_np = image_np[:, :, None]
    image_np = np.concatenate([image_np, image_np, image_np], axis=2)
    canny_image = Image.fromarray(image_np)

    pipe.set_ip_adapter_scale(0.8)

    # 3. Generate
    output = pipe(
        prompt,
        image=canny_image,
        ip_adapter_image=style_image,
        controlnet_conditioning_scale=1.5,
        guidance_scale=7.5,
        num_inference_steps=20
    ).images[0]

    return output

# --- 5. THE LCEL CHAIN (The Logic) ---
# Logic: Retrieve Style Path -> Pass everything to Generator
def retrieve_step(prompt):
    docs = retriever.invoke(prompt)
    return docs[0].metadata["path"]

# The Chain Definition
rag_chain = RunnablePassthrough.assign(
    style_path=lambda x: retrieve_step(x["prompt"])
) | RunnableLambda(generation_node)


# --- 6. FASTAPI SERVER ---
app = FastAPI()

@app.post("/generate")
async def generate_design(file: UploadFile = File(...), prompt: str = Form(...)):
    try:
        # 1. Read Image
        image_data = await file.read()
        sketch_image = Image.open(io.BytesIO(image_data)).convert("RGB")
        
        # 2. RAG RETRIEVAL (We grab the path explicitly here)
        # We assume rag_chain has a step that returns metadata, 
        # OR we just use the retriever directly to get the name for the header.
        docs = retriever.invoke(prompt)
        best_style_path = docs[0].metadata["path"]
        best_style_name = docs[0].page_content # We stored the name here in the previous fix
        
        print(f"RAG Picked: {best_style_name}")

        # 3. GENERATION
        result_image = rag_chain.invoke({"prompt": prompt, "sketch": sketch_image})

        # 4. Return Image + Header
        img_byte_arr = io.BytesIO()
        result_image.save(img_byte_arr, format='PNG')
        
        # We send the "best_style_name" back so Streamlit knows what was picked
        return Response(
            content=img_byte_arr.getvalue(), 
            media_type="image/png",
            headers={"X-Retrieved-Style": best_style_name} 
        )

    except Exception as e:
        traceback.print_exc()
        return Response(content=str(e), status_code=500)

# --- 7. START SERVER ---
public_url = ngrok.connect(addr="127.0.0.1:8000", bind_tls=True).public_url
print(f"\n✅ LANGCHAIN API URL: {public_url} \n")

def start_server():
    uvicorn.run(app, port=8000, host="127.0.0.1")

threading.Thread(target=start_server, daemon=True).start()

2025-12-08 08:26:49.793197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765182409.815067     526 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765182409.821728     526 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

Building LangChain VectorStore...
Ingested: Cyberpunk
Ingested: Ghibli
Ingested: Sketch
Ingested: Industrial
Loading SD Pipeline...


/tmp/ipykernel_526/1575132915.py:111: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]


✅ LANGCHAIN API URL: https://overclose-georgann-soporifically.ngrok-free.dev 



INFO:     Started server process [526]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
